In [1]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch import Tensor
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
from torchvision import datasets, transforms
from torchvision.io import read_image

# Read the preprocessed file names

In [9]:
# Transforming a Folder of images into csv File
try:
    DATA_DIR = os.path.join('training_data')
except:
    raise RuntimeError('Error for the training_data directory')

try:
    train_folder = os.path.join(DATA_DIR, 'train_data')
except:
    raise RuntimeError('Error for the train_data directory')


#print(os.path.abspath('training_data'))
#print(train_folder)

train_files = [i for i in os.listdir(train_folder) if i.endswith('.png')]
Image_name = [i for i in list(train_files)]

i=0
while i< len(Image_name):
    Image_name[i] = Image_name[i][0:24]
    i=i+1
    
    
df = pd.DataFrame()
df['img_files'] = [os.path.join(train_folder, i) for i in train_files]
df['img_name'] = Image_name
df.head()
                               
#print(Image_name)

,img_files,img_name
0,training_data/train_data/preproc_test_image017...,preproc_test_image017249
1,training_data/train_data/preproc_test_image018...,preproc_test_image018393
2,training_data/train_data/preproc_test_image017...,preproc_test_image017881
3,training_data/train_data/preproc_test_image017...,preproc_test_image017117
4,training_data/train_data/preproc_test_image018...,preproc_test_image018230


In [50]:
# Open the dataset 
controlInput_df = pd.read_csv(os.path.join(DATA_DIR, 'dataset.csv'), names=['x', 'y', 'img_name'], header=None)
print(controlInput_df.head)

#print(controlInput_df.loc[[3],['y']])
# Create variables to better compare with the 

<bound method NDFrame.head of        x   y                  img_name
0    240  10  preproc_test_image015405
1    250  10  preproc_test_image015406
2    255  10  preproc_test_image015416
3    265  10  preproc_test_image015417
4    245  10  preproc_test_image015428
..   ...  ..                       ...
263  254   2  preproc_test_image024418
264  254   1  preproc_test_image024419
265  253   1  preproc_test_image024426
266  255   1  preproc_test_image024430
267  255   4  preproc_test_image024431

[268 rows x 3 columns]>


In [44]:
# class CustomDataset(Dataset):

#     def __init__(self, csvFile,ImageName, transform=None):
#         """
#         Args:
#             csv_file (string): Path to the csv file with annotations.
#             root_dir (string): Directory with all the images.
#             transform (callable, optional): Optional transform to be applied
#                 on a sample.
#         """
#         self.csv_file = csvFile
#         #self.root_dir = root_dir
#         self.transform = transform
#         Image_name = ImageName

#     def __len__(self):
#         return len(self.csv_file)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

#         img_name = Image_name(idx)
#         print(Image_name(2))
#         image = io.imread(img_files(idx))
#         i=0
#         while i< len(self.csv_file)+1:
#             if img_name == controlInput_df.loc[[i],['img_name']]:
#                 xPosition = controlInput_df.loc[[i],['x']]
#                 yPosition = controlInput_df.loc[[i],['y']]
#                 sample = {'imageName': image_name, 'image': image,'x':xPosition,'y':yPosition}
#             #landmarks = np.array([landmarks])
#             #landmarks = landmarks.astype('float').reshape(-1, 2)
#             i =i+1
        

#         if self.transform:
#             sample = self.transform(sample)

#         return sample

In [47]:
custom_dataset = CustomDataset(csvFile=controlInput_df,ImageName=Image_name)
print(custom_dataset)

In [49]:

i=0
while i< len(self.csv_file)+1:
    if img_name == controlInput_df.loc[[i],['img_name']]:
        xPosition = controlInput_df.loc[[i],['x']]
        yPosition = controlInput_df.loc[[i],['y']]

TypeError: 'list' object is not callable

# Read and convert the BeBOT trajectory data

In [3]:
BeBot_df = pd.read_csv(os.path.join(DATA_DIR, 'data_pix.csv'), names=['x', 'y', 'img_name'], header=None)
print(BeBot_df.head())

def read_bebot_data(bebot_df):
    """Reads the BeBOT dataframe and converts it to a dictionary
    
    RETURNS
        dict('img_name', np.array) - where 'img_name' is a string containing the image name associated with the data and np.array is a numpy array of [x0, ..., xn, y0, ..., yn, tf]
    """
    data = []
    for traj in bebot_df.iterrows():
        tmp = [float(i) for i in traj[1]['x'].strip('[]').split()]
        tmp += [float(i) for i in traj[1]['y'].strip('[]').split()]
#         tmp += [traj[1]['tf']]
        tmp += [i.strip(' []') for i in traj[1]['img_name'].split()]
        data.append(tmp)
        
    data_dict = dict(zip([i[-1] for i in data], [np.array(i[:-1]) for i in data]))
    return data_dict

bebot_data = read_bebot_data(BeBot_df)
bebot_data['image221308']

                                                   x  \
0  [250.         250.         250.         250.  ...   
1  [250.         249.         248.         247.  ...   
2  [250.        251.        252.        253.     ...   
3  [250.         251.         252.         253.  ...   
4  [250.        252.        254.        256.     ...   

                                                   y     img_name  
0  [480.         433.         386.         339.  ...  image221307  
1  [480.         433.5        387.         340.5 ...  image221308  
2  [480.         433.         386.         339.  ...  image221309  
3  [480.         432.8        385.6        338.4 ...  image221310  
4  [480.         433.         386.         339.  ...  image221311  


array([250.        , 249.        , 248.        , 247.        ,
       246.        , 245.        , 244.        , 243.        ,
       242.        , 241.        , 239.99999735, 480.        ,
       433.5       , 387.        , 340.5       , 294.        ,
       247.5       , 201.        , 154.5       , 108.        ,
        61.5       ,  14.99999738])

Create databases

Below we create the datasets. The variable true_trajs contains the real trajectories generated using BeBOT (i.e. this would be the y_train dataset).

In [4]:
true_trajs = DataLoader(TensorDataset(Tensor(np.array(list(bebot_data.values())))), batch_size=10, shuffle=True)

In [5]:
class generator(nn.Module):
    # Network Architecture based on infoGAN (https://arxiv.org/abs/1606.03657)
    def __init__(self, img_channels=3, input_dim=100, output_dim=3, input_size=300, base_size=64):
        super(generator, self).__init__()
        self.img_channels = img_channels #3 channels for RGB
        self.input_dim = input_dim #number of batches
        self.output_dim = output_dim #output
        self.input_size = input_size #size of the image
        self.base_size = base_size 
        
        self.conv = nn.Sequential(
            # See https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html for shape info
            nn.Conv2d(self.img_channels, self.base_size, 4, stride=2, padding=1), # Output: base_size x 150 x 150
            nn.LeakyReLU(0.2),
            nn.Conv2d(self.base_size, 2*self.base_size, 4, stride=2, padding=1), # Output: 2*base_size x 75 x 75
            nn.BatchNorm2d(2*self.base_size),
            nn.LeakyReLU(0.2),
        )
        
        self.cn = nn.Sequential(
            nn.Linear(self.input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 2*self.base_size * (self.input_size // 4) * (self.input_size // 4)),
            nn.BatchNorm1d(2*self.base_size * (self.input_size // 4) * (self.input_size // 4)),
            nn.ReLU(),
        )

        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(2*self.base_size, self.base_size, 4, 2, 1),
            nn.BatchNorm2d(self.base_size),
            nn.ReLU(),
            nn.ConvTranspose2d(self.base_size, self.output_dim, 4, 2, 1),
            nn.Tanh(),
        )
        initialize_weights(self)

        self.l1 = nn.Linear(2,10)
        self.l2 = nn.Linear(10, 10)
        self.l3 = nn.Linear(10,3)
        self.fc = nn.Linear(6, 6)

    def forward(self, img, pt):
#         x = self.cn(input)
#         print(x.shape)
#         x = x.view(-1, 2*self.base_size, (self.input_size // 4), (self.input_size // 4))
#         print(x.shape)
#         x = self.deconv(x)
#         print(x.shape)
#         y = self.l1(y)
#         print(x.shape)
#         y = self.l2(y)
#         print(x.shape)
#         y = self.l3(y)
#         print(x.shape)
#         z = torch.cat([x,y],1)
#         print(x.shape)
#         z = self.fc(z)
        x = self.conv(img)
        print(x.shape)
        x = x.view(-1, 2*self.base_size * (self.input_size // 4) * (self.input_size // 4))
        print(x.shape)

        return x

In [6]:
class discriminator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    def __init__(self, input_dim=1, output_dim=1, input_size=32,base_size=64):
        super(discriminator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = 28
        self.base_size = base_size

        self.conv = nn.Sequential(
            nn.Conv2d(self.input_dim, self.base_size, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(self.base_size, 2*self.base_size, 4, 2, 1),
            nn.BatchNorm2d(2*self.base_size),
            nn.LeakyReLU(0.2),
        )
        self.fc = nn.Sequential(
            nn.Linear(2*self.base_size * (self.input_size // 4) * (self.input_size // 4), 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, self.output_dim),  # Note: no activation at the output.
        )
        initialize_weights(self)

    def forward(self, input):
        x = self.conv(input)
        x = x.view(-1, 2*self.base_size * (self.input_size // 4) * (self.input_size // 4))
        x = self.fc(x)

        return x

In [7]:
def initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.ConvTranspose2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()

In [8]:
true_trajs.dataset.__len__()

219

In [9]:
class GAN():
    def __init__(self, params):
        """Generative Adversarial Network class for learning how to control a self driving car with Bernstein polynomials
        
        Parameters
        ----------
        params : dict
            Dictionary containing the parameters of the problem.
                * max_epochs - Maximum number of epochs
                * z_dim
                * base_size
                * lr_g - Generator learning rate
                * lr_d - Discriminator learning rate
                * beta1
                * beta2
        Returns
        -------
        numpy.array
            1D vectory containing the initial guess for the optimizer. See above
            description for the layout of the vector.
        """
        # parameters
        self.epoch = params['max_epochs']
        self.sample_num = 100
        self.batch_size = 300
        self.input_size = 300
        self.z_dim = params['z_dim']
        self.base_size = params['base_size']

#         load dataset
#         self.data_loader = torch.utils.data.DataLoader(train_files, 
#                                                batch_size=self.batch_size, 
#                                                shuffle=True)
#         self.create_data_loader()
        data = true_trajs.__iter__().__next__()[0]

        # initialization of the generator and discriminator
        self.G = generator(input_dim=self.z_dim, output_dim=data.shape[1], input_size=self.input_size,base_size=self.base_size).cuda()
        self.D = discriminator(input_dim=data.shape[1], output_dim=1, input_size=self.input_size,base_size=self.base_size).cuda()
        self.G_optimizer = optim.Adam(self.G.parameters(), lr=params['lr_g'], betas=(params['beta1'], params['beta2']))
        self.D_optimizer = optim.Adam(self.D.parameters(), lr=params['lr_d'], betas=(params['beta1'], params['beta2']))
        
        # initialization of the loss function
        self.BCE_loss = nn.BCELoss().cuda()
        self.MSE_loss = nn.MSELoss().cuda()
        
#     def create_data_loader(self):
#         self.data_loader = torch.utils.data.DataLoader(true_trajs, 
#                                                        batch_size=self.batch_size, 
#                                                        shuffle=True)
        
    #--------------------------------------------------------------------------------------------------------
    # Fucntion to train the GAN, where you alternate between the training of the genenator and discriminator
    def train(self):

       # Setting empty arrays for storing the losses
        self.train_hist = {}
        self.train_hist['D_loss'] = []
        self.train_hist['G_loss'] = []

        # Setting up the labels for real and fake images
        self.y_real_, self.y_fake_ = torch.ones(self.batch_size, 1).cuda(), torch.zeros(self.batch_size, 1).cuda()
        
        print('training start!!')

        # Epoch loops
        for epoch in range(self.epoch):
            epoch_start_time = time.time()


            for iter, (x_) in enumerate(true_trajs):
#                 if iter == true_trajs.dataset.__len__() // self.batch_size:
#                     print(true_trajs.dataset.__len__() // self.batch_size)
#                     print(iter)
#                     print('break')
#                     break

                # Generate random noise to push through the generator   
                x_ = x_.cuda()

                # update D network
                for i in range(Ninner):
                    self.D_optimizer.zero_grad()
                    x_ = x_  + 0.025 * torch.randn(x_.shape).cuda()
                    D_real = self.D(x_)
                    D_real_loss = self.BCE_loss(D_real, self.y_real_)

                    G_ = self.G(z_)
                    D_fake = self.D(G_)
                    D_fake_loss = self.BCE_loss(D_fake, self.y_fake_)

                    D_loss = D_real_loss + D_fake_loss
                    D_loss.backward()
                    self.D_optimizer.step()    

                # update G network
                for i in range(Ninner):
                    self.G_optimizer.zero_grad()
                    G_ = self.G(z_)
                    D_fake = self.D(G_)
                    G_loss = self.BCE_loss(D_fake, self.y_real_)

                    G_loss.backward()
                    self.G_optimizer.step()

                self.train_hist['D_loss'].append(D_loss.item())
                self.train_hist['G_loss'].append(G_loss.item())

                # Print iterations and losses
                if ((iter + 1) % 50) == 0:
                    print("Epoch: [%2d] [%4d/%4d] D_loss: %.8f, G_loss: %.8f" %
                    ((epoch + 1), (iter + 1), true_trajs.dataset.__len__() // self.batch_size, D_loss.item(), G_loss.item()))
                
#             # Visualize results
#             with torch.no_grad():
#                 visualize_results(self)

        print("Training finished!")


In [10]:
params = {'beta1': 0.5, 'beta2': 0.999,'lr_g':0.0009,'lr_d':0.0009,'max_epochs':5} 
params['z_dim'] =2
params['base_size'] = 64 

gan = GAN(params)
gan.train()

AssertionError: Torch not compiled with CUDA enabled